# Variance Inflation Factor bestimmen

In diesem Notebook bringen wir Klarheit in das Konstrukt "Variance Inflation Factor". Der VIF ist ein Zeichen für Kollinearität und ist definiert als 

$$VIF = \frac{1}{1-R^2_{X_j|X_{-j}}}$$

wobei $R^2_{X_j|X_{-j}}$ das $R^2$ von der Regression der $j$-ten Variable gegen alle anderen Variablen der Prädiktormatrix $X$ ist.

Wir betrachten für das VIF also **nur** die Prädiktoren, nicht die Responsevariable, denn wir wollen ja Kollinearitäten zwischen den Prädiktoren erkennen.

Der kleinste mögliche Wert für das VIF ist 1, dann ist absolut keine Kollinearität vorhanden. 

In der Praxis gilt: Ein VIF größer als 5 oder 10 zeigt ein problematisches Ausmaß an Kollinearität.

Ist das $R^2_{X_j|X_{-j}}$ nahe 1, dann liegt Kollinearität vor und das VIF ist sehr groß. 

Das VIF kann auch als Diagonalelemente der inversen Korrelationsmatrix bestimmt werden.

In [4]:
# Import dstools (absolute path required, please change to your systems settings)
import importlib
import sys

path = '/dstools-master/dstools/__init__.py'
name = 'dstools'

spec = importlib.util.spec_from_file_location(name, path)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)

## Daten laden

In [5]:
import pandas as pd
import numpy as np

from dstools.datasets import bodyfat
from dstools.tools import quality, vif

In [6]:
df = bodyfat()
df.convert(unit="metric")
df.head()

D:\dstools-master/dstools\datasets\bodyfat.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['Weight', 'Height']] = df[['Weight', 'Height']].applymap(lambda x: round(x, 2))
D:\dstools-master/dstools\datasets\bodyfat.py:90: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[to_convert] = df[to_convert].applymap(lambda x: round(x * 0.39370, 2))
D:\dstools-master/dstools\datasets\bodyfat.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[weights] = df[weights].applymap(lambda x: round(x * 0.45359237, 2))
D:\dstools-master/dstools\datasets\bodyfat.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[lengths] = df[lengths].applymap(lambda x: round(x / 0.39370, 2))


,Density,Percent,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,69.97,172.09,36.20,93.09,85.19,94.49,59.00,37.31,21.89,32.00,27.41,17.09
1,1.0853,6.1,22,78.58,183.52,38.51,93.60,83.01,98.70,58.70,37.31,23.39,30.51,28.91,18.21
2,1.0414,25.3,22,69.85,168.28,34.01,95.81,87.91,99.21,59.59,38.89,24.00,28.80,25.20,16.61
3,1.0751,10.4,26,83.80,183.52,37.39,101.80,86.41,101.19,60.10,37.31,22.81,32.41,29.39,18.21
4,1.0340,28.7,24,83.57,180.98,34.39,97.31,100.00,101.91,63.20,42.19,24.00,32.21,27.71,17.70


In [7]:
# Get X and y for Linear Regression
X, y = df.for_regression()

### VIF mittels inverser Korrelationsmatrix

Wir können die inverse Matrix direkt aus unserem Pandas-Dataframe berechnen:

In [5]:
np.diag(np.linalg.inv(X.corr()))

array([ 2.24776988, 33.51170689,  1.67378493,  4.32439027,  9.4614085 ,
       11.77919266, 14.79902778,  7.77200665,  4.61118844,  1.90609216,
        3.62195761,  2.19216283,  3.37953898])

### VIF mit dstools berechnen

In [6]:
?vif

In [7]:
vif(X, threshold=1)

,columns,vif
1,Weight,33.511707
6,Hip,14.799028
5,Abdomen,11.779193
4,Chest,9.461408
7,Thigh,7.772007
8,Knee,4.611188
3,Neck,4.324390
10,Biceps,3.621958
12,Wrist,3.379539
0,Age,2.247770


In [8]:
# default value for threshold is 5
vif(X)

,columns,vif
1,Weight,33.511707
6,Hip,14.799028
5,Abdomen,11.779193
4,Chest,9.461408
7,Thigh,7.772007


### VIF mit der Funktion variance_inflation_factor

Das Paket statsmodels enthält eine Funktion "variance_inflation_factor", die allerdings zwei Besonderheiten hat:

1. Als Input benötigen wir numpy-Arrays
1. Der Prädiktormatrix muss ein Intercept-Term hinzugefügt werden

Grund für die erste Bedingung: Intern verwendet die Funktion ein Slicing, das nicht mit Pandas-Dataframes funktioniert.

Grund für die zweite Bedingung: Die Funktion verwendet OLS(), welches die Intercept-Spalte nicht wie smf.ols() automatisch hinzufügt, sondern explizit braucht.

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [10]:
# Transformation in ein Numpy-Array und Hinzufügen der Einser Spalte
dfn = X.to_numpy()
datamat = np.append(np.ones((dfn.shape[0],1)),dfn, axis=1)
datamat

array([[ 1.  , 23.  , 69.97, ..., 32.  , 27.41, 17.09],
       [ 1.  , 22.  , 78.58, ..., 30.51, 28.91, 18.21],
       [ 1.  , 22.  , 69.85, ..., 28.8 , 25.2 , 16.61],
       ...,
       [ 1.  , 72.  , 84.71, ..., 31.29, 27.2 , 18.01],
       [ 1.  , 72.  , 86.52, ..., 30.51, 29.39, 19.81],
       [ 1.  , 74.  , 94.12, ..., 33.71, 30.  , 20.9 ]])

In [11]:
# VIF Berechnen:
for i in range(1,13):
    vf = variance_inflation_factor(datamat,i)
    print(vf)

2.247769881801335
33.511706892726444
1.673784929009302
4.324390274242906
9.461408499100358
11.779192662747784
14.799027782700893
7.772006650787686
4.611188442615579
1.9060921554220234
3.621957608139303
2.1921628275750025


### VIF als Regression berechnen

Wir berechnen den VIF exemplarische für die Variable "Weight"

In [12]:
import statsmodels.formula.api as smf

In [13]:
results = smf.ols('Weight ~ Age + Height + Neck + Chest + Abdomen + Hip + Thigh + Knee + Ankle + Biceps + Forearm + Wrist', data=df).fit()
vif = 1/(1-results.rsquared)
vif

np.float64(33.511706892726444)